## Import

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Load file and specify the board name

In [ ]:
con = sqlite3.connect('../ETROC-Data/BaselineHistory.sqlite')
df = pd.read_sql_query("SELECT * from baselines", con)

## Print the board name

In [ ]:
column_values = df['chip_name']

# Print the unique elements in the column
unique_elements = column_values.unique()
for element in unique_elements:
    print(element)

## Print the saved notes for dedicate board name

In [ ]:
chip_name = 'ET2_W36_IP5-17'
column_values = df[(df['chip_name'] == chip_name)]['note']

# Print the unique elements in the column
unique_elements = column_values.unique()
for element in unique_elements:
    print(element)

## Select the data based on the board name and note

In [ ]:
base_df = df[(df['chip_name'] == chip_name) & (df['note'] == 'LowBiasCurrent HVoff CblOff ET2_W36_IP5-17')]
base_df

## Convert data to 2D 16x16 numpy array

In [ ]:
# Create a pivot table to reshape the DataFrame
pivot_table = base_df.pivot(index='row', columns='col', values='baseline')
baseline = pivot_table.values

pivot_table = base_df.pivot(index='row', columns='col', values='noise_width')
noisewidth = pivot_table.values

## Draw BL and NW

In [ ]:
fig = plt.figure(dpi=200, figsize=(10,10))
gs = fig.add_gridspec(1,2)

ax0 = fig.add_subplot(gs[0,0])
ax0.set_title("BL (DAC LSB)", size=8)
img0 = ax0.imshow(baseline, interpolation='none')
ax0.set_aspect("equal")
ax0.invert_xaxis()
ax0.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img0, cax=cax, orientation="vertical")

ax1 = fig.add_subplot(gs[0,1])
ax1.set_title("NW (DAC LSB)", size=8)
img1 = ax1.imshow(noisewidth, interpolation='none')
ax1.set_aspect("equal")
ax1.invert_xaxis()
ax1.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img1, cax=cax, orientation="vertical")

for x in range(16):
    for y in range(16):
        ax0.text(x,y,f"{baseline.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
        ax1.text(x,y,f"{noisewidth.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")